In [2]:
import torch
import numpy as np
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader, TensorDataset

In [3]:
data = np.loadtxt("/content/cardiovascular_diseases_dv3.csv", delimiter=';', skiprows=1)
X_data = data[:, :11]
Y_data = data[:, 11]
print(X_data.shape)
print(Y_data.shape)

(68783, 11)
(68783,)


In [4]:
print(X_data)
print(Y_data)

[[ 50.   2. 168. ...   0.   0.   1.]
 [ 55.   1. 156. ...   0.   0.   1.]
 [ 52.   1. 165. ...   0.   0.   0.]
 ...
 [ 52.   2. 183. ...   0.   1.   0.]
 [ 61.   1. 163. ...   0.   0.   0.]
 [ 56.   1. 170. ...   0.   0.   1.]]
[0. 1. 1. ... 1. 1. 0.]


In [5]:
# Normalizar los datos
scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)

In [6]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

# Convertir los datos a tensores de PyTorch
X_train_tensor = torch.from_numpy(X_train).float()
Y_train_tensor = torch.from_numpy(Y_train).long()# Usar LongTensor para etiquetas de clasificación binaria
X_test_tensor = torch.from_numpy(X_test).float()
Y_test_tensor = torch.from_numpy(Y_test).long()# Usar LongTensor para etiquetas de clasificación binaria

In [7]:

# Asegurarte de que Y_train_tensor sea un tensor 1D (por ejemplo, si contiene etiquetas [0, 1, 0, 1, ...])
Y_train_tensor = Y_train_tensor.view(-1,1)

print(Y_train_tensor)

tensor([[1],
        [1],
        [0],
        ...,
        [1],
        [0],
        [1]])


In [8]:
# Convertir las etiquetas de entrenamiento a tipo Float
Y_train_tensor = Y_train_tensor.float()

In [9]:
# Definir el modelo de regresión logística
class LogisticRegressionModel(torch.nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = torch.nn.Linear(input_dim, 1)
        self.sigmoid = torch.nn.Sigmoid()#sigmoid que representa una decisión entre dos clases.la función sigmoide se utiliza en la capa de salida del modelo para transformar la salida en una probabilidad de pertenencia a una de las dos clases.

    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

In [10]:
# Inicializar el modelo
input_dim = X_train.shape[1] #numerod de columnas
model = LogisticRegressionModel(input_dim)

In [11]:
# Definir la función de pérdida y el optimizador
criterion = torch.nn.BCELoss()  # Binary Cross Entropy Loss para clasificación binaria
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)#definimos el optimizador

In [12]:
torch.cuda.is_available()
#No se puede conectar con el backend de GPU
#En estos momentos no puedes conectarte a una GPU debido a los límites de uso de Colab

False

In [13]:
# Entrenamiento del modelo
epochs = 100
log_each = 10
l = []

model.train()
for e in range(1, epochs + 1):
    # Forward
    y_pred = model(X_train_tensor)
    #print(y_pred)
    # Calcular la pérdida
    loss = criterion(y_pred, Y_train_tensor)
    l.append(loss.item())

    # Reiniciar los gradientes
    optimizer.zero_grad()

    # Retropropagación
    loss.backward()

    # Actualizar los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/100 Loss 0.72469
Epoch 20/100 Loss 0.72406
Epoch 30/100 Loss 0.72343
Epoch 40/100 Loss 0.72281
Epoch 50/100 Loss 0.72219
Epoch 60/100 Loss 0.72158
Epoch 70/100 Loss 0.72096
Epoch 80/100 Loss 0.72036
Epoch 90/100 Loss 0.71975
Epoch 100/100 Loss 0.71915


In [18]:
#NO PUEDO CONCECTAR CON LA GPU DEBIDO AL USO EXCESIBO

from sklearn.metrics import accuracy_score #libreria para calcular la precisión de las predicciones
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)


def evaluate(x):
    model.eval()#poner el modelo en modo evaluacion
    y_pred = model(x) # realiza una inferencia en el modelo usando el tensor de entrada X para obtenera las predicciones
    y_probas = softmax(y_pred)#para obtener la probabilidades de clase, Convierte las puntuaciones del modelo en distribuciones de porbabilidad
    return torch.argmax(y_probas, axis=1)#encontrar la clase con la probabilidad mas alta para cada modelo

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())#Realiza la evaluación del modelo en el conjunto de datos de prueba
accuracy_score(Y_train_tensor, y_pred.cpu().numpy())

RuntimeError: ignored